<a href="https://colab.research.google.com/github/Rohith-Bollam/new-project/blob/main/Instagram_Post_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
!pip install langgraph langchain

In [95]:
!pip install langchain-google-genai

  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [97]:
from google.colab import userdata
API_KEY=userdata.get('GOOGLE_API_KEY_1')

In [98]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=API_KEY)

In [100]:
from langgraph.graph import MessageGraph,END
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate

In [101]:
writer_prompt = ChatPromptTemplate.from_template(
    "Write a Instagram post about this topic: {topic}"
)

writer_chain = writer_prompt | model

reviewer_prompt = ChatPromptTemplate.from_template(
    "Review the following Instagram post and suggest improvements: {post}"
)

reviewer_chain = reviewer_prompt | model

In [102]:
graph = MessageGraph()

def writer_node(state):
  print(state)
  topic = state[-1].content
  print(topic)
  post = writer_chain.invoke({"topic": topic})
  print(type(post))
  return state + [AIMessage(content = post.content)]

def reviewer_node(state):
  print(state)
  post = state[-1].content
  review = reviewer_chain.invoke({"post": post})
  return state + [AIMessage(content = review.content)]

graph.add_node("Writer", writer_node)
graph.add_node("Reviewer", reviewer_node)
graph.set_entry_point("Writer")

def condition_check(state):
  print("From condition_check")
  if len(state)>=5:
    print("Printing from If")
    return END
  else:
    print("Printing from Else")
    return "Reviewer"

graph.add_conditional_edges("Writer", condition_check)
graph.add_edge("Reviewer", "Writer")

app = graph.compile()

In [103]:
response = app.invoke([HumanMessage(content = "Best side hustles for students")])
for message in response:
  print(message.content)

[HumanMessage(content='Best side hustles for students', additional_kwargs={}, response_metadata={}, id='2cc8f6be-6a31-46b4-b730-51c9f00d2260')]
Best side hustles for students
<class 'langchain_core.messages.ai.AIMessage'>
From condition_check
Printing from Else
[HumanMessage(content='Best side hustles for students', additional_kwargs={}, response_metadata={}, id='2cc8f6be-6a31-46b4-b730-51c9f00d2260'), AIMessage(content="Okay, here's an Instagram post draft about the best side hustles for students, with a few options to tailor it to your specific style:\n\n**Option 1: Short & Sweet**\n\n```\n📚💰 Level up your student life with these side hustles! ⬇️\n\n*   **Freelance Writing/Editing:** Flex those English skills!\n*   **Online Tutoring:** Share your knowledge and get paid!\n*   **Social Media Management:**  Become a content creator for businesses.\n*   **Delivery Driver:** Set your own hours and explore the city.\n*   **Virtual Assistant:** Help businesses with administrative tasks.\n\n